# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    ER = 0
    data['Ri'] = (data['Price in 1 Year'] - P) / P
    
    for index, row in data.iterrows():
        ER += row['Probability'] * row['Ri']
    return ER

In [4]:
exp_return(data, 40)

0.075

Write the function ```volatility```, which calculates the volatility of the stock, given the current stock price ```P```.

In [5]:
def volatility(data, P):
    Var = 0
    ER = exp_return(data, P)
    data['Ri'] = (data['Price in 1 Year'] - P) / P
    
    for index, row in data.iterrows():
        Var += row['Probability'] * (row['Ri'] - ER)**2
    return Var**0.5

In [6]:
volatility(data, 40)

0.21301408404140793

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [7]:
def interval(data, P):
    ER = exp_return(data, P)
    SD = volatility(data, P)
    N = len(data)
    SE = SD / N**0.5
    return [ER - 1.96*SE, ER + 1.96*SE]

In [8]:
interval(data, 40)

[-0.16604812797447732, 0.31604812797447734]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [9]:
def exp_return_capm(beta, market_return, rf):
    mkt_risk_premium = market_return - rf
    return rf + beta * mkt_risk_premium

In [10]:
exp_return_capm(-0.6,0.1, 0.02)

-0.028

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [11]:
def clean_data(path):
    df = pd.read_csv(path)
    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df.dropna(inplace=True)
    df = df.groupby("CUSIP").filter(lambda x: len(x) >= 500).apply(lambda x: x)
    return df

In [12]:
df = clean_data("monthlycrsp_mod6.csv")
df

PERMNO        date     CUSIP       RET
9027      10065  1970-01-30  00621210 -0.144375
9028      10065  1970-02-27  00621210  0.105882
9029      10065  1970-03-31  00621210  0.017857
9030      10065  1970-04-30  00621210 -0.078947
9031      10065  1970-05-29  00621210 -0.020952
...         ...         ...       ...       ...
3393558   84129  2022-08-31  98970110  0.016315
3393559   84129  2022-09-30  98970110 -0.075777
3393560   84129  2022-10-31  98970110  0.021235
3393561   84129  2022-11-30  98970110  0.005583
3393562   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [13]:
def avg_return(data):
    data = data.groupby('CUSIP')
    cusips = data.first().index
    avg_returns = data['RET'].mean()
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips.values,
        "Average Return" : avg_returns.values
    })
    
    return output

In [14]:
def security_volatilities(data):
    ER = avg_return(data)
    data = data.merge(ER, on='CUSIP')
    data['Deviation^2'] = (data['RET'] - data['Average Return'])**2
    
    data = data.groupby('CUSIP')
    cusips = data.first().index
    SD = (data['Deviation^2'].agg(lambda x: x.sum() / (x.count() - 1)))**0.5

    output = pd.DataFrame(data = {
        "CUSIP" : cusips.values,
        "Volatility" : SD.values
    })
    
    return output

In [15]:
ER = avg_return(df)
ER

CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]

In [16]:
SD = security_volatilities(df)
SD

CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [17]:
def correlation(ret_data, vol_data):
    data = ret_data.merge(vol_data, on='CUSIP')
    return data['Average Return'].corr(data['Volatility'])

In [18]:
correlation(ER, SD)

0.44837090727059625